# Slate Star Codex Reader Survey 2018

This is the data wrangling notebook to accompany my analysis. It's probably not of interest to you unless a) you're a novice data scientist and want to learn some new stuff, or b) you want to see my methodology to understand the strengths and weaknesses of my approach.

TL;DR:

- I dummify the multiple choice questions. This means I convert them into multiple yes/no (1/0) questions so we can mathematically analyze them.

- For questions that allowed an "Other" response, I removed responses that appeared fewer than 3 times, and dummified the rest.

- I removed responses that used strings of letters when numbers were expected. For instance, writing "ten" instead of "10." This happened often enough that's it's not worth the effort to try fixing them all. We just need a stricter survey next time!

- I use machine learning to try predicting missing values. This is largely successful.

**If you enjoy this, let's connect on LinkedIn:**

**https://www.linkedin.com/in/vincefavilla/**

I'm a psychologist and machine learning engineer. Also, I'm in the SF Bay Area and looking for a job outside of academia. :)

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_excel('ssc2018public_original.xlsx')
# data = pd.read_csv('ssc2018public_original.csv', encoding='latin-1', low_memory=False)

data.shape

(7298, 187)

In [3]:
# Open-ended categoricals; people could enter their own text as "Other" (bad for data analysis!)
# We'll limit them to the most common responses.

to_simplify = '''Country
State
ReligiousDenomination
ReligiousBackground
PoliticalAffiliation
PoliticalChange
PoliticalChangeSSC
Favoriteblog
SSCBenefit
SSCChangeMind
PoliticalDisagreementI
PoliticalDisagreementII'''.split('\n')

# My rule of thumb is "3 is a pattern." So let's remove responses that appear 
# fewer than 3 times and replace them with "Other"
for i in to_simplify:
    value_counts = data[i].str.lower().value_counts()
    value_counts = value_counts[value_counts > 2]
    data[i] = np.where(data[i].str.lower().isin(value_counts.index), data[i].str.title(), 'Other')

In [4]:
# Good categoricals -- need no additional processing
to_dummify = '''Race
Sex
Gender
SexualOrientation
WorkStatus
Profession
Degree
ReligiousViews
ReligiousChange
ReligiousChangeII
MoralViews
LengthofTime
AmericanParties
PoliticalDisagreementIII
TheSystem
IncomeStatus
IncomeChildhood
Cryptocurrency
GWWC
CharityCareer
Depression
Anxiety
OCD
Eatingdisorder
Alcoholism
Drugaddiction
Borderline
Bipolar
Autism
ADHD
Schizophrenia
Adderall
Vegetarian
BirthdateEnding
Harassment1
Harassment2
Harassment3
Harassment4
NaziCategory
MarriageCategory
Handedness
BreatheThrough
Placebo
PreviousSurveys
Mask1
Mask2
Mask3
ParenthesesPalindrome
MapRiddle
SurgeonRiddle
Navon1
Navon2
Sundown
Dancer
Dancer2
SquaresCircles
Tables
Cookies
Reversal
LOC1
LOC2
LOC3
CRTM
Wason
Referrals
PostsRead
Comment
Subreddit
Discord
HiddenOpenThreads
Meetup
PatreonI'''.split('\n')

# Begin dummification!
for i in to_dummify:
    dums = pd.get_dummies(data[i], prefix=i)
    data = pd.concat([data,dums], axis=1)

# We've now simplified these, sot they're ready to dummify too.
for i in to_simplify:
    dums = pd.get_dummies(data[i], prefix=i)
    data = pd.concat([data,dums], axis=1)

In [5]:
data['ChangeOverTenYears']

0       10.0
1        2.0
2        9.0
3        8.0
4        NaN
5       10.0
6        1.0
7        2.0
8        NaN
9        7.0
10       1.0
11       3.0
12       8.0
13      10.0
14       9.0
15       4.0
16       8.0
17       4.0
18      10.0
19       5.0
20       8.0
21       6.0
22      10.0
23       7.0
24       7.0
25       NaN
26      10.0
27       6.0
28       7.0
29       8.0
        ... 
7268     4.0
7269     8.0
7270     7.0
7271     3.0
7272    10.0
7273     6.0
7274    10.0
7275    10.0
7276     5.0
7277     9.0
7278     NaN
7279     6.0
7280    10.0
7281     7.0
7282    10.0
7283     3.0
7284     3.0
7285     NaN
7286     9.0
7287     4.0
7288     6.0
7289    10.0
7290     7.0
7291     NaN
7292     8.0
7293    10.0
7294     6.0
7295     5.0
7296     3.0
7297     7.0
Name: ChangeOverTenYears, Length: 7298, dtype: float64

In [6]:
# I'm going to attempt to turn these questions into numerical scales.
# It's not perfect, but I think it's good enough to justify doing.

scales = '''Asexuality
Relationshipstyle
EducationComplete
LWID
EAID
SJID'''.split('\n')

relationship = np.zeros_like(data['Relationshipstyle'])
relationship = np.where(data['Relationshipstyle'] == 'Prefer monogamous', 1, relationship)
relationship = np.where(data['Relationshipstyle'] == 'Prefer polyamorous', 0, relationship)
relationship = np.where(data['Relationshipstyle'] == 'Other', 0.5, relationship)
relationship = np.where(data['Relationshipstyle'] == 'Uncertain / no preference', 0.5, relationship)
data['Monogamy'] = pd.Series(relationship)

data['EducationComplete'] = np.where(data['EducationComplete'] == 'Yes', 1, 0)

def yes_no_sorta(col):
    new_col = np.zeros_like(data[col])
    new_col = np.where(data[col] == 'Yes', 1, new_col)
    new_col = np.where(data[col] == 'No', 0, new_col)
    new_col = np.where(data[col] == 'Sorta', 0.5, new_col)
    return pd.Series(new_col)

data['LWID'] = yes_no_sorta('LWID')
data['EAID'] = yes_no_sorta('EAID')
data['SJID'] = yes_no_sorta('SJID')
data['Asexuality'] = yes_no_sorta('Asexuality')

In [7]:
data['Relationship'] = np.where(data['RelationshipStatus'] == 'Single', 0, 1)
del data['RelationshipStatus']

# I'll make a wild assumption that among people with
# 4+ children, the average is 4.5.
data['Children'] = data.Children.str.replace(r'4+', '4.5')

We're going to convert the following columns into numeric values, removing any response that contains non-numeric characters. We're going to lose information the process: imagine someone responding "about 1400" vs. "1400" for their SAT score; the former is now considered an invalid answer. Due to their sheer amount of data, we have few other options. The survey can be improved next time by forcing numeric responses and instructing people to estimate when necessary.

In [8]:
data['IQ'] = pd.to_numeric(data['IQ'], errors='coerce')
data['Age'] = pd.to_numeric(data['Age'], errors='coerce')
data['SATscoresoutof1600'] = pd.to_numeric(data['SATscoresoutof1600'], errors='coerce')
data['SATscoresoutof2400'] = pd.to_numeric(data['SATscoresoutof2400'], errors='coerce')
data['Income'] = pd.to_numeric(data['Income'], errors='coerce')
data['Charity'] = pd.to_numeric(data['Charity'], errors='coerce')
data['OlderBrothers'] = pd.to_numeric(data['OlderBrothers'], errors='coerce')
data['YoungerBrothers'] = pd.to_numeric(data['YoungerBrothers'], errors='coerce')
data['YoungerSisters'] = pd.to_numeric(data['YoungerSisters'], errors='coerce')
data['CRT1'] = pd.to_numeric(data['CRT1'], errors='coerce')
data['CRT2'] = pd.to_numeric(data['CRT2'], errors='coerce')
data['CRT3'] = pd.to_numeric(data['CRT3'], errors='coerce')
data['AutismSpectrumTest'] = pd.to_numeric(data['AutismSpectrumTest'], errors='coerce')
data['GenderRoleTestM'] = pd.to_numeric(data['GenderRoleTestM'], errors='coerce')
data['GenderRoleTestF'] = pd.to_numeric(data['GenderRoleTestF'], errors='coerce')
data['Gender2M'] = pd.to_numeric(data['Gender2M'], errors='coerce')
data['Gender2F'] = pd.to_numeric(data['Gender2F'], errors='coerce')
data['Children'] = pd.to_numeric(data['Children'], errors='coerce')

for i in data.columns:
    if 'BigFive' in i:
        data[i] = pd.to_numeric(data[i], errors='coerce')
        data[i] = np.where(data[i] > 100, data[i].mean(), data[i])
        data[i] = np.where(data[i] < 0, data[i].mean(), data[i])

# This question was reversed, so let's fix it
data['GlobalWarming'] = 6 - data['GlobalWarming']

# Neuroticism was actually reported as emotional stability, so let's reverse it as well
data['BigFiveN'] = 100 - data['BigFiveN']

# Combine these two
data['State_Washington Dc'] = data['State_Washington Dc'] + data['State_Washington, Dc']
del data['State_Washington, Dc']

Now let's clean up the data and remove liars and outliers (heh). For the most part, I'm pretty trusting of responses, but just to give you an idea of what we have:

In [9]:
data[(data['Income'] > 10000000)][['Age', 'Gender', 'Country', 'Income']]

,Age,Gender,Country,Income
5015,37.0,M (cisgender),United States,15000000.0
6046,24.0,M (cisgender),Brazil,60000000.0


It's perfectly reasonable that a 37-year-old American is making \$15 million a year, but I'm a little skeptical of the 24-year-old Brazilian making \$60 million. It's not impossible, but it's going to skew our data regardless, so I'm removing him. (If you're the rich Brazilian, drop me a line!)

In [10]:
# Income less than $60M
data = data[(data.Income.isnull()) | (data.Income < 60000000)]

# I doubt anyone younger than 12 reads SSC
data = data[(data.Age.isnull()) | (data.Age > 11)]

# People who entered impossible scores
data = data[(data.SATscoresoutof1600.isnull()) | (data.SATscoresoutof1600 <= 1600)]
data = data[(data.SATscoresoutof2400.isnull()) | (data.SATscoresoutof2400 <= 2400)]

# Some IQ scores seem a liiittle implausible.
data = data[(data.IQ.isnull()) | ((data.IQ < 175) & (data.IQ > 70))]

In [11]:
# I'll be ignoring the following features, either because
# they're too open-ended, cause errors, or are too specific to SSC

to_delete = '''Adderall4
BlogReferrals
PostReferrals
PatreonII
PatreonIII
PatreonIV
Classified
PoliticalChangeDescription
SurveyTime
Adderall2
Adderall3
Meetup2
Favoriteblog
Favoritepost'''.split('\n')

for i in to_delete:
    del data[i]

I don't always like missing data, so we're going to use some machine learning to infer the missing values. I'll make datasets available both with and without this technique, so you can use whichever version you prefer.

In [12]:
data = data.select_dtypes(include=[np.number])
data = data.astype(float)
data.columns = data.columns.str.replace('[', '(')
data.columns = data.columns.str.replace(']', ')')
data.columns = data.columns.str.replace('<', 'lessthan')
data.to_csv('ssc2018public_cleaned.csv')

# Number of missing values per question
missing = pd.DataFrame(data.isnull().sum().sort_values(ascending=False))
missing[missing > 0].dropna()

,0
SATscoresoutof2400,5785.0
ChildrenHappiness,5448.0
IQ,5258.0
SATscoresoutof1600,4952.0
AnxietyScale,4402.0
AnxietyFIXED,3238.0
BigFiveE,2940.0
BigFiveC,2933.0
BigFiveA,2933.0
BigFiveO,2932.0


In [13]:
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor

# Get null columns
null_cols = data.isnull().sum()
null_cols = list(null_cols[null_cols != 0].index)

for i in null_cols:
    x = data.fillna(data.median())
    y = x.pop(i)
    
    xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2)
    reg = XGBRegressor()
    reg.fit(xtrain, ytrain)      
    pred = reg.predict(xtest)
    r2 = r2_score(ytest, pred)
    
    # If we can reasonably predict these values, do so
    if r2 > .20:
        print('regressing', i, 'with an r2 of', round(r2, 3))
        data['predicted'] = reg.predict(data.fillna(data.median()).drop([i], axis=1))
        data[i] = np.where(data[i].isnull(), data['predicted'], data[i])
        del data['predicted']
    
    # Otherwise, just take the median
    else:
        print('averaging', i)
        data[i] = data[i].fillna(data[i].median())

regressing Age with an r2 of 0.699
regressing GenderConformity with an r2 of 0.385
regressing Children with an r2 of 0.446
averaging ChildrenHappiness
regressing GenderThoughts with an r2 of 0.26
averaging IQ
averaging SATscoresoutof1600
averaging SATscoresoutof2400
averaging Percentile
regressing SSCFavorability with an r2 of 0.395
regressing SSCAgreement with an r2 of 0.368
regressing Rationalistfavorability with an r2 of 0.301
regressing PoliticalSpectrum with an r2 of 0.715
averaging PoliticalInterest
regressing GlobalWarming with an r2 of 0.513
regressing Immigration with an r2 of 0.457
regressing MinimumWage with an r2 of 0.547
regressing GayMarriage with an r2 of 0.506
regressing Feminism with an r2 of 0.499
regressing HumanBiodiversity with an r2 of 0.312
regressing BasicIncome with an r2 of 0.396
regressing DonaldTrump with an r2 of 0.509
regressing PaulRyan with an r2 of 0.399
regressing BarackObama with an r2 of 0.515
regressing BernieSanders with an r2 of 0.54
averaging Inc

In [14]:
data.to_csv('ssc2018public_inferred.csv')

In [15]:
data.IQ.min()

90.0